## Default Factories

- defualt value,如果使用method()產生不同的default value值,是無法完成的,如下範例

In [1]:
from datetime import datetime, timezone
from pydantic import BaseModel,Field
import random

# defualt value,如果使用method()產生不同的default value值,是無法完成的,如下範例

class Scores(BaseModel):
    chinese:int = random.randint(50, 100)

s1 = Scores()
s2 = Scores()
s3 = Scores()

s1, s2, s3

(Scores(chinese=94), Scores(chinese=94), Scores(chinese=94))

In [3]:
#要解決這個問題就必需使用default factories

class Scores(BaseModel):
    chinese:int = Field(default_factory=lambda:random.randint(50,100))

s1 = Scores()
s2 = Scores()
s3 = Scores()

s1, s2, s3

(Scores(chinese=95), Scores(chinese=83), Scores(chinese=99))